# FVA: Compare flux distribution ACSBG

Strategy: fixe growth rate to its experimental value and minimize photon uptake

### Directories

In [ ]:
GEM_folder = "..\\..\\..\\.."

In [ ]:
modelfile = "model\\iMgadit23.json"

### Imports

In [ ]:
import json
from math import log
import numpy as np
from os.path import join as __join
import pandas as pd
import time

import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis

### Model

In [ ]:
def load_model_add_constraints(cobra_model_file):
    cobra_model = cobra.io.load_json_model(cobra_model_file)
    cobra_model.reactions.EX_photon_abs_.lower_bound = -331
    return(cobra_model)

# FVA Simualtions

### Conditions

In [ ]:
cond_1 = "BIOMASS_biomass_WT526_c"

In [ ]:
cond_2 = "BIOMASS_biomass_MgACSBG31_c"

In [ ]:
dict_experimental_growth = {"BIOMASS_biomass_WT526_c": 3.04E-02 ,
                            "BIOMASS_biomass_MgACSBG31_c": 1.50E-02}

### 1) FVA WT

In [ ]:
iMgadit23 = load_model_add_constraints(__join(GEM_folder,modelfile))
iMgadit23.objective = "EX_photon_abs_"
for rxn in iMgadit23.reactions:
    if rxn.id[0:16] == "BIOMASS_biomass_":
        if rxn.id == cond_1:
            rxn.lower_bound = dict_experimental_growth[rxn.id]
            rxn.upper_bound = dict_experimental_growth[rxn.id]
            print(rxn.id, "bounds = ",rxn.lower_bound, rxn.upper_bound)
        else:
            rxn.lower_bound = 0
            rxn.upper_bound = 0
            print(rxn.id, "bounds = ",rxn.lower_bound, rxn.upper_bound)
df_FVA1 = flux_variability_analysis(iMgadit23,fraction_of_optimum=1)
df_FVA1.columns = ["FVA min WT","FVA max WT"]

In [ ]:
df_FVA1

In [ ]:
del iMgadit23

In [ ]:
iMgadit23 = load_model_add_constraints(__join(GEM_folder,modelfile))
iMgadit23.objective = "EX_photon_abs_"
for rxn in iMgadit23.reactions:
    if rxn.id[0:16] == "BIOMASS_biomass_":
        if rxn.id == cond_2:
            rxn.lower_bound = dict_experimental_growth[rxn.id]
            rxn.upper_bound = dict_experimental_growth[rxn.id]
            print(rxn.id, "bounds = ",rxn.lower_bound,rxn.upper_bound)
        else:
            rxn.lower_bound = 0
            rxn.upper_bound = 0
            print(rxn.id, "bounds = ",rxn.lower_bound,rxn.upper_bound)
df_FVA2 = flux_variability_analysis(iMgadit23,fraction_of_optimum=1)
df_FVA2.columns = ["FVA min acsbg31","FVA max acsbg31"]

In [ ]:
res_FVA = pd.concat([df_FVA1,df_FVA2], axis=1)

In [ ]:
res_FVA

In [ ]:
res_FVA.to_csv("Res_FVA_raw.csv",sep=";")